[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GabbyTab/boofun/blob/main/notebooks/lecture9_dnf_restrictions.ipynb)

# Lecture 9: DNFs & Random Restrictions

**Topics**: DNF formulas, Random restrictions, Switching Lemma preview

**O'Donnell Chapters**: 3.3, 4.1, 4.3  
**Based on lecture notes by: Qinggao Hong**  
**Notebook by: Gabriel Taboada**

---

## Key Concepts

| Term | Definition |
|------|------------|
| **DNF** | Disjunctive Normal Form (OR of ANDs) |
| **Width** | Maximum clause size in DNF |
| **Random Restriction** | Fix random subset of variables, leave rest "free" |
| **$p$-random restriction** | Each variable is free with probability $p$ |

**Main result**: Width-$w$ DNFs have total influence $\leq w$ (Theorem 9.3)

In [1]:
# Install/upgrade boofun (required for Colab)
!pip install --upgrade boofun -q

import boofun as bf
print(f"BooFun version: {bf.__version__}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


/Users/gabrieltaboada/Library/Python/3.9/lib/python/site-packages/boofun/core/errormodels.py:21: UserWarning: uncertainties library not available - some error models disabled
  warnings.warn("uncertainties library not available - some error models disabled")


/Users/gabrieltaboada/Library/Python/3.9/lib/python/site-packages/boofun/quantum/__init__.py:22: UserWarning: Qiskit not available - quantum features limited
  warnings.warn("Qiskit not available - quantum features limited")


BooFun version: 1.1.1


In [2]:
import numpy as np
import boofun as bf
from boofun.analysis import SpectralAnalyzer
from boofun.analysis.restrictions import (
    random_restriction,
    apply_restriction,
    restriction_shrinkage,
)

import warnings
warnings.filterwarnings('ignore')

---
## 1. DNFs and Fourier Concentration

**Tribes** is the canonical balanced DNF: $k$ "tribes" of $w$ variables each, where a tribe is satisfied if ALL its members are +1.

$$\text{Tribes}_{k,w}(x) = \bigvee_{i=1}^{k} \bigwedge_{j \in T_i} x_j$$

In [3]:
# Tribes function is a canonical DNF
tribes = bf.tribes(3, 9)  # 3 tribes of 3 variables each
print(f"Tribes(3,9): {tribes.n_vars} variables (3 tribes x 3 vars each)")
print(f"Total Influence: {tribes.total_influence():.4f}")

# Spectral weight by degree
weights = tribes.spectral_weight_by_degree()
print(f"\nFourier weight by degree (W_k = Sum|S|=k f_hat(S)^2):")
for k, w in weights.items():
    bar = '#' * int(w * 40)
    print(f"  k={k}: {w:.4f} {bar}")

# Spectral concentration
analyzer = SpectralAnalyzer(tribes)
print(f"\nSpectral concentration (cumulative weight):")
for d in range(1, tribes.n_vars + 1):
    conc = analyzer.spectral_concentration(d)
    if conc > 0.99:
        print(f"  W^<={d}[f] = {conc:.4f}  <- 99% of weight by degree {d}")
        break
    elif d <= 5:
        print(f"  W^<={d}[f] = {conc:.4f}")

Tribes(3,9): 9 variables (3 tribes x 3 vars each)
Total Influence: 1.7227

Fourier weight by degree (W_k = Sum|S|=k f_hat(S)^2):
  k=0: 0.1155 ####
  k=1: 0.3297 #############
  k=2: 0.3499 #############
  k=3: 0.1507 ######
  k=4: 0.0349 #
  k=5: 0.0151 
  k=6: 0.0035 
  k=7: 0.0005 
  k=8: 0.0001 
  k=9: 0.0000 

Spectral concentration (cumulative weight):
  W^<=1[f] = 0.4452
  W^<=2[f] = 0.7951
  W^<=3[f] = 0.9458
  W^<=4[f] = 0.9807
  W^<=5[f] = 0.9958  <- 99% of weight by degree 5


---
## 2. Random Restrictions

**Definition 9.6**: A restriction $(J, z)$ fixes variables outside $J$ to values $z$, leaving $J$ "free".

**$p$-random restriction**: Each variable is independently free with probability $p$.

The restricted function $f_{J,z}: \{\pm 1\}^J \to \{\pm 1\}$ has fewer variables.

In [4]:
# Demonstrate random restrictions
f = bf.majority(7)
print(f"Original: Majority-7 ({f.n_vars} variables)")
print(f"Total influence: {f.total_influence():.4f}")

rng = np.random.default_rng(42)

# Apply a p=0.5 random restriction
p = 0.5
rho = random_restriction(f.n_vars, p, rng)
print(f"\nRestriction (p={p}): {rho}")
print(f"  Fixed vars: {dict(rho.fixed)}")
print(f"  Free vars: {sorted(rho.free)}")

# Apply the restriction
f_restricted = apply_restriction(f, rho)
print(f"\nRestricted function: {f_restricted.n_vars} free variables")
print(f"Restricted total influence: {f_restricted.total_influence():.4f}")

Original: Majority-7 (7 variables)
Total influence: 2.1875

Restriction (p=0.5): 1*10*11
  Fixed vars: {0: 1, 2: 1, 3: 0, 5: 1, 6: 1}
  Free vars: [1, 4]

Restricted function: 2 free variables
Restricted total influence: 0.0000


---
## 3. The "High-Pass Filter" Effect

**Key intuition**: Random restrictions act like a **high-pass filter** on Fourier weights!

From Proposition 9.7, after a random $p$-restriction:
- Low-degree coefficients "survive" with probability $p^{|S|}$
- High-degree coefficients are more likely to be killed

**Result**: The restricted function has weight shifted toward **higher relative degrees**.

In [5]:
# Demonstrate the high-pass filter effect
def print_weight_bars(weights, n_vars, title, max_bars=30):
    """Print ASCII bar chart of Fourier weights."""
    print(f"\n{title} (n={n_vars})")
    print("-" * 50)
    max_w = max(weights.values()) if weights else 1
    for k in sorted(weights.keys()):
        w = weights[k]
        bar_len = int((w / max_w) * max_bars) if max_w > 0 else 0
        bar = '#' * bar_len
        print(f"  k={k}: {w:.4f} {bar}")

# Compare original vs restricted Tribes
tribes = bf.tribes(3, 9)
rng = np.random.default_rng(42)

print("=" * 60)
print("HIGH-PASS FILTER EFFECT: Restrictions shift weight to higher degrees")
print("=" * 60)

# Original
orig_weights = tribes.spectral_weight_by_degree()
print_weight_bars(orig_weights, tribes.n_vars, "ORIGINAL Tribes(3,9)")

# Apply a single restriction and show the effect
p = 0.5
rho = random_restriction(tribes.n_vars, p, rng)
f_restricted = apply_restriction(tribes, rho)

print(f"\nRestriction (p={p}): {rho}")
print(f"Free variables: {sorted(rho.free)}")

if f_restricted.n_vars > 0:
    rest_weights = f_restricted.spectral_weight_by_degree()
    print_weight_bars(rest_weights, f_restricted.n_vars, "RESTRICTED function")
    
    # Show the shift in relative terms
    print(f"\n-> Notice: degree-0 (constant) weight increased from {orig_weights.get(0,0):.4f} to {rest_weights.get(0,0):.4f}")
    print(f"-> Weight is now spread across fewer degrees, shifted relatively higher!")

# Average over many restrictions
print("\n" + "=" * 60)
print("AVERAGE over 50 random restrictions:")
print("=" * 60)

avg_weights = {}
total_free = 0
num_samples = 50

for _ in range(num_samples):
    rho = random_restriction(tribes.n_vars, p, rng)
    f_rho = apply_restriction(tribes, rho)
    if f_rho.n_vars > 0:
        total_free += f_rho.n_vars
        for k, w in f_rho.spectral_weight_by_degree().items():
            avg_weights[k] = avg_weights.get(k, 0) + w / num_samples

avg_free = total_free / num_samples
print_weight_bars(avg_weights, f"~{avg_free:.1f}", f"AVERAGE restricted (p={p})")

print(f"\n-> Random restrictions 'filter out' low-degree structure!")

HIGH-PASS FILTER EFFECT: Restrictions shift weight to higher degrees

ORIGINAL Tribes(3,9) (n=9)
--------------------------------------------------
  k=0: 0.1155 #########
  k=1: 0.3297 ############################
  k=2: 0.3499 ##############################
  k=3: 0.1507 ############
  k=4: 0.0349 ##
  k=5: 0.0151 #
  k=6: 0.0035 
  k=7: 0.0005 
  k=8: 0.0001 
  k=9: 0.0000 

Restriction (p=0.5): 1*10*111*
Free variables: [1, 4, 8]

RESTRICTED function (n=3)
--------------------------------------------------
  k=0: 1.0000 ##############################
  k=1: 0.0000 
  k=2: 0.0000 
  k=3: 0.0000 

-> Notice: degree-0 (constant) weight increased from 0.1155 to 1.0000
-> Weight is now spread across fewer degrees, shifted relatively higher!

AVERAGE over 50 random restrictions:

AVERAGE restricted (p=0.5) (n=~4.5)
--------------------------------------------------
  k=0: 0.3572 #########################
  k=1: 0.4179 ##############################
  k=2: 0.1869 #############
  k=3: 0.03

---
## 4. Shrinkage Under Restrictions

**Lemma 9.9**: $\mathbb{E}[\text{Inf}[f_{J,z}]] = p \cdot \text{Inf}[f]$

Random restrictions **shrink** complexity measures:
- Total influence scales by $p$
- Decision tree depth decreases dramatically (Switching Lemma)

In [6]:
# Demonstrate shrinkage
print("Shrinkage Under Random Restrictions")
print("=" * 55)

functions = {
    "Tribes(3,9)": bf.tribes(3, 9),
    "Majority-9": bf.majority(9),
    "Parity-8": bf.parity(8),
}

for name, f in functions.items():
    stats = restriction_shrinkage(f, p=0.5, num_samples=100)
    
    print(f"\n{name}:")
    print(f"  Original DT depth: {stats['original_dt_depth']}")
    print(f"  Avg restricted DT depth: {stats['avg_restricted_dt_depth']:.2f}")
    print(f"  Depth shrinkage: {stats['depth_shrinkage_factor']:.2f}x")
    print(f"  Became constant: {stats['constant_fraction']*100:.0f}% of restrictions")

Shrinkage Under Random Restrictions



Tribes(3,9):
  Original DT depth: 9
  Avg restricted DT depth: 2.52
  Depth shrinkage: 0.28x
  Became constant: 0% of restrictions



Majority-9:
  Original DT depth: 9
  Avg restricted DT depth: 4.23
  Depth shrinkage: 0.47x
  Became constant: 0% of restrictions

Parity-8:
  Original DT depth: 8
  Avg restricted DT depth: 3.79
  Depth shrinkage: 0.47x
  Became constant: 1% of restrictions


---
## Summary

| Result | Statement |
|--------|----------|
| **Thm 9.3** | Width-$w$ DNF has $\text{Inf}[f] \leq w$ |
| **Prop 9.2** | Width-$w$ DNF is $\varepsilon$-concentrated on degree $O(w/\varepsilon)$ |
| **Lemma 9.9** | $\mathbb{E}[\text{Inf}[f_{J,z}]] = p \cdot \text{Inf}[f]$ |

**Switching Lemma** (next lecture): After $p$-restriction, width-$w$ DNF becomes depth-$O(\log(1/p))$ decision tree with high probability.

### boofun API

```python
from boofun.analysis.restrictions import random_restriction, apply_restriction

rho = random_restriction(n, p)      # Generate p-random restriction
f_rho = apply_restriction(f, rho)   # Apply to get restricted function
f.spectral_weight_by_degree()       # W_k[f] for each degree k
restriction_shrinkage(f, p)         # Statistics on shrinkage
```